In [6]:
import numpy as np
import pandas as pd
from hlf_funciones import *
from hlf_variables import *


In [3]:
#VARIABLES.PY
DIMENSION_TABLERO = 10
BARCOS = {
    "Barco1": 4,
    "Barco2": 3,
    "Barco3": 3,
    "Barco4": 2,
    "Barco5": 2,
    "Barco6": 2,
    "Barco7": 1,
    "Barco8": 1,
    "Barco9": 1,
    "Barco10": 1
}

In [6]:
#CLASES.PY
import numpy as np
#from hlf_variables import DIMENSION_TABLERO, BARCOS

class Tablero:
    def __init__(self, id_jugador):
        self.id_jugador = id_jugador
        self.dimensiones = (DIMENSION_TABLERO, DIMENSION_TABLERO)
        self.barcos = BARCOS
        self.tablero = np.zeros(self.dimensiones, dtype=int)
        self.tablero_disparos = np.zeros(self.dimensiones, dtype=int)

    def colocar_barco(self, x, y, eslora, orientacion):
        if orientacion == 'horizontal':
            for i in range(eslora):
                self.tablero[x + i, y] = 1
        elif orientacion == 'vertical':
            for i in range(eslora):
                self.tablero[x, y + i] = 1

    def colocar_barcos(self):
        for barco, eslora in self.barcos.items():
            colocado = False
            while not colocado:
                x = np.random.randint(0, DIMENSION_TABLERO)
                y = np.random.randint(0, DIMENSION_TABLERO)
                orientacion = np.random.choice(['horizontal', 'vertical'])
                if self.validar_posicion_barco(x, y, eslora, orientacion):
                    self.colocar_barco(x, y, eslora, orientacion)
                    colocado = True

    def validar_posicion_barco(self, x, y, eslora, orientacion):
        if orientacion == 'horizontal':
            if x + eslora > DIMENSION_TABLERO:
                return False
            for i in range(eslora):
                if self.tablero[x + i, y] != 0:
                    return False
        elif orientacion == 'vertical':
            if y + eslora > DIMENSION_TABLERO:
                return False
            for i in range(eslora):
                if self.tablero[x, y + i] != 0:
                    return False
        return True

    def disparo_coordenada(self, coordenada):
        x, y = self.letras_coordenada(coordenada)
        if self.tablero[x, y] == 1:
            print("Tocado!")
            self.tablero_disparos[x, y] = 1
            if self.barco_hundido(x, y):
                print("Barco hundido!")
        else:
            print("Agua!")
            self.tablero_disparos[x, y] = -2

    def barco_hundido(self, x, y):
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                cont = 0
                nx, ny = x, y
                while 0 <= nx < DIMENSION_TABLERO and 0 <= ny < DIMENSION_TABLERO and self.tablero[nx, ny] == 1:
                    cont += 1
                    nx += dx
                    ny += dy
                if cont == 0:
                    continue
                elif cont == 1:
                    return True
                elif cont > 1:
                    return False

    def letras_coordenada(self, coordenada):
        letras = 'ABCDEFGHIJ'
        x = letras.index(coordenada[0].upper())
        y = int(coordenada[1:]) - 1
        return x, y

    def imprimir_tablero(self):
        print("Tablero de", self.id_jugador)
        print("  ", "  ".join(str(i) for i in range(1, DIMENSION_TABLERO + 1)))
        letras = 'ABCDEFGHIJ'
        for i in range(DIMENSION_TABLERO):
            row = ''.join(str(cell) for cell in self.tablero_disparos[i])
            print(letras[i], row.replace('0', ' - ').replace('1', ' X ').replace('-2', ' O ')) 
            

In [7]:
#FUNCIONES.PY
import numpy as np
#from hlf_variables import DIMENSION_TABLERO

def generar_coordenada_aleatoria():
    letras = 'ABCDEFGHIJ'
    x = np.random.randint(0, DIMENSION_TABLERO)
    y = np.random.randint(0, DIMENSION_TABLERO)
    return letras[x] + ' ' + str(y + 1)

def validar_coordenada(coordenada):
    letras = 'ABCDEFGHIJ'
    if len(coordenada) < 2:
        return False
    if coordenada[0].upper() not in letras:
        return False
    if not coordenada[1:].isdigit():
        return False
    if int(coordenada[1:]) < 1 or int(coordenada[1:]) > DIMENSION_TABLERO:
        return False
    return True

def juego_terminado(tablero):
    return np.sum(tablero.tablero) == 0

In [8]:
#MAIN.PY
def main():
    print("¡Bienvenido a Batalla Naval!")

    # Inicializar tableros de ambos jugadores
    tablero_jugador = Tablero("Jugador")
    tablero_maquina = Tablero("Máquina")
    tablero_jugador.colocar_barcos()
    tablero_maquina.colocar_barcos()

    # Ciclo principal del juego
    while True:
        # Turno del jugador
        print("\nTurno del Jugador:")
        tablero_maquina.imprimir_tablero()
        coordenada_jugador = input("Ingrese coordenada para disparar: ")
        print("El jugador dispara a la coordenada:", coordenada_jugador)
        if not validar_coordenada(coordenada_jugador):
            print("Coordenada inválida. Inténtalo de nuevo.")
            continue
        tablero_maquina.disparo_coordenada(coordenada_jugador)
        if juego_terminado(tablero_maquina):
            print("¡Has ganado! ¡Felicidades!")
            break

        # Turno de la máquina
        print("\nTurno de la Máquina:")
        tablero_jugador.imprimir_tablero()
        coordenada_maquina = generar_coordenada_aleatoria()
        print("La Máquina dispara a la coordenada:", coordenada_maquina)
        tablero_jugador.disparo_coordenada(coordenada_maquina)
        if juego_terminado(tablero_jugador):
            print("¡La Máquina ha ganado! ¡Inténtalo de nuevo!")
            break

# if __name__ == "__main__":
#     main()

In [9]:
main()

¡Bienvenido a Batalla Naval!

Turno del Jugador:
Tablero de Máquina
   1  2  3  4  5  6  7  8  9  10
A  -  -  -  -  -  -  -  -  -  - 
B  -  -  -  -  -  -  -  -  -  - 
C  -  -  -  -  -  -  -  -  -  - 
D  -  -  -  -  -  -  -  -  -  - 
E  -  -  -  -  -  -  -  -  -  - 
F  -  -  -  -  -  -  -  -  -  - 
G  -  -  -  -  -  -  -  -  -  - 
H  -  -  -  -  -  -  -  -  -  - 
I  -  -  -  -  -  -  -  -  -  - 
J  -  -  -  -  -  -  -  -  -  - 
El jugador dispara a la coordenada: 
Coordenada inválida. Inténtalo de nuevo.

Turno del Jugador:
Tablero de Máquina
   1  2  3  4  5  6  7  8  9  10
A  -  -  -  -  -  -  -  -  -  - 
B  -  -  -  -  -  -  -  -  -  - 
C  -  -  -  -  -  -  -  -  -  - 
D  -  -  -  -  -  -  -  -  -  - 
E  -  -  -  -  -  -  -  -  -  - 
F  -  -  -  -  -  -  -  -  -  - 
G  -  -  -  -  -  -  -  -  -  - 
H  -  -  -  -  -  -  -  -  -  - 
I  -  -  -  -  -  -  -  -  -  - 
J  -  -  -  -  -  -  -  -  -  - 
El jugador dispara a la coordenada: B6
Agua!

Turno de la Máquina:
Tablero de Jugador
   